In [ ]:
CATALOG = dbutils.widgets.get("CATALOG")
COMPLAINT_LAKEBASE_INSTANCE_NAME = f"{CATALOG}complaintmanager"

In [ ]:
%pip install databricks-sdk --upgrade

In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
import uuid
import time

import sys
sys.path.append('../utils')
from uc_state import add

# Initialize the Workspace client
w = WorkspaceClient()

UNIQUE = str(uuid.uuid4())[:8]

# Create a database instance
instance = w.database.create_database_instance(
    DatabaseInstance(
        name=f"{COMPLAINT_LAKEBASE_INSTANCE_NAME}",
        capacity="CU_1"
    )
)

print(f"Created database instance: {instance.name}")
add(CATALOG, "databaseinstances", {"name": instance.name, "type": "complaint"})
print(f"Connection endpoint: {instance.response.read_write_dns}")

# block until it's actually ready
print("Waiting for database instance to be available...")
while True:
    if str(w.database.get_database_instance(instance.name).state) == "DatabaseInstanceState.AVAILABLE":
        print(str(w.database.get_database_instance(instance.name).state))
        break
    time.sleep(10)

In [ ]:
# create a database `caspers_complaints` in the lakebase instance
# register it to UC as concat of CATALOG-INSTANCE_NAME (params)

from databricks.sdk.service.database import DatabaseCatalog

catalog = w.database.create_database_catalog(
    DatabaseCatalog(
        name=f"{COMPLAINT_LAKEBASE_INSTANCE_NAME}",                   # Name of the UC catalog to create
        database_instance_name=COMPLAINT_LAKEBASE_INSTANCE_NAME, # Name of the database instance
        database_name="caspers_complaints",         # Name of the Postgres database to register (and optionally create)
        create_database_if_not_exists=True    # Create the database if it doesn't exist
    )
)
print(f"Created new database and catalog: {catalog.name}")

add(CATALOG, "databasecatalogs", catalog)

In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy

# Initialize the Workspace client
w = WorkspaceClient()

# Create a synced table in a standard UC catalog
synced_table = w.database.create_synced_database_table(
    SyncedDatabaseTable(
        name=f"{CATALOG}.complaints.pg_complaint_responses",  # Full three-part name
        database_instance_name=instance.name,  # Required for standard catalogs
        logical_database_name="caspers_complaints",  # Required for standard catalogs
        spec=SyncedTableSpec(
            source_table_full_name=f"{CATALOG}.complaints.complaint_responses",
            primary_key_columns=["complaint_id"],
            scheduling_policy=SyncedTableSchedulingPolicy.CONTINUOUS,
            create_database_objects_if_missing=True,  # Create database/schema if needed
            new_pipeline_spec=NewPipelineSpec(
                storage_catalog="storage_catalog",
                storage_schema="storage_schema"
            )
        ),
    )
)
print(f"Created synced table: {synced_table.name}")
add(CATALOG, "pipelines", synced_table.data_synchronization_status)